In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import random
from sklearn.metrics import confusion_matrix, mean_squared_error, log_loss, accuracy_score
import time
from tensorflow.keras.models import model_from_json
from tensorflow.keras import layers
from tensorflow.keras import optimizers

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
model_n = 17
opt = optimizers.Adam()

In [3]:
# load json and create model
model_path = 'results/NNmodel_{}_with_1.0data.json'.format(model_n)
print(model_path)
weights_path = 'results/NNmodel_{}_with_1.0data_weights.hdf5'.format(model_n)

json_file = open(model_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(weights_path)
print("Loaded model from disk")

# evaluate loaded model 

loaded_model.compile(loss='mean_squared_error', optimizer=opt,
                     metrics=['mean_squared_error'])

W0826 02:37:05.467292 139994449573632 deprecation.py:506] From /data/home/t-chepan/env/newlab/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0826 02:37:05.468248 139994449573632 deprecation.py:506] From /data/home/t-chepan/env/newlab/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0826 02:37:05.469382 139994449573632 deprecation.py:506] From /data/home/t-chepan/env/newlab/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:97: calling Ze

results/NNmodel_17_with_1.0data.json
Loaded model from disk


In [4]:
Xdev = np.load('data/Xdev.npy')
ydev = np.load('data/ydev.npy')

In [5]:
pred_dev = loaded_model.predict(Xdev)
dev_mse = mean_squared_error(ydev, pred_dev)
dev_mse

0.0728153816103259

## Use permutation to analyze the importance of features

In [8]:
n_example, n_features = Xdev.shape
n_features

489

In [132]:
# a = np.arange(12).reshape((3,4))
# a

In [133]:
# a[:,1] = np.random.permutation(a[:,1])

In [134]:
# a = np.arange(12).reshape((3,4))
# np.random.permutation(a[:,1])
# a

In [135]:
# len(a[0])

In [136]:
# a = np.arange(12).reshape((3,4))

# for i in range(len(a[0])):
#     permut_a = np.zeros(np.shape(a))
#     permut_a[:] = a
#     permut_a[:,i] = np.random.permutation(a[:,i])
#     print(permut_a)

## trial with fake settings

In [105]:
t1 = time.time()

permut_X = Xdev

permut_X[:,990] = np.random.permutation(Xdev[:,990])
pred_permute = loaded_model.predict(permut_X)
mse_permute = mean_squared_error(ydev, pred_permute)
print(mse_permute)

t2 = time.time()
print('used time in {} seconds'.format(t2-t1))

0.08164757472081949
used time in 0.2709469795227051 seconds


In [107]:
randsample = 3
n_features = 5

In [110]:
permutsample = np.zeros((randsample, n_features))
for trying in range(randsample):
    t1 = time.time()
    print('This is sample {}'.format(trying))
    for i in range(n_features):
        permut_X = np.zeros(np.shape(Xdev))
        permut_X[:] = Xdev
        permut_X[:,-i] = np.random.permutation(Xdev[:,-i])
        permut_pred = loaded_model.predict(permut_X)
        permut_mse = mean_squared_error(ydev, permut_pred)
        permutsample[trying, -i] = permut_mse
    t2 = time.time()
    print('Use time in {} seconds'.format(t2-t1))
    print('*' * 20)


This is sample 0
Use time in 1.5444622039794922 seconds
********************
This is sample 1
Use time in 1.551302194595337 seconds
********************
This is sample 2
Use time in 1.545614242553711 seconds
********************


In [111]:
print(permutsample)

[[0.08166467 0.07619916 0.09034454 0.13429679 0.08338546]
 [0.08168226 0.08278596 0.08930428 0.13448791 0.08589154]
 [0.08167277 0.08031207 0.088329   0.17884274 0.08598124]]


In [112]:
# np.save('results/permut_dev_results.npy', permutsample)

## Complete running the code within .py file and load the results

In [9]:
## train within the .py file. Now load the data
permutsample = np.load('results/permut_dev_results.npy')
permutsample.shape

(10, 489)

In [10]:
# print mean and standard deviation of error
errperformance = np.zeros((n_features, 2))
for i in range(n_features):
    errperformance[i,0] = np.mean(permutsample[:,i])
    errperformance[i,1] = np.std(permutsample[:,i])
errperformance[:,0] = errperformance[:,0] - dev_mse

print(errperformance)

[[ 5.49338912e-06  9.00049611e-06]
 [-1.38777878e-17  1.38777878e-17]
 [ 3.04768909e-04  7.93978547e-05]
 [-1.38777878e-17  1.38777878e-17]
 [-1.38777878e-17  1.38777878e-17]
 [-1.38777878e-17  1.38777878e-17]
 [-6.76314778e-06  7.25226455e-06]
 [-1.38777878e-17  1.38777878e-17]
 [ 4.30471520e-06  4.34314669e-06]
 [ 7.19077752e-05  7.31645698e-06]
 [-1.38777878e-17  1.38777878e-17]
 [ 4.16490678e-04  1.79278945e-03]
 [ 1.53777702e-03  2.22782493e-03]
 [ 4.40822879e-06  3.93906107e-06]
 [-1.00265524e-05  4.27524475e-06]
 [ 5.32733230e-05  6.79915239e-06]
 [ 4.20900640e-05  8.57078152e-05]
 [ 3.08944508e-06  5.46721314e-06]
 [ 8.08344517e-03  9.70570982e-03]
 [-1.38777878e-17  1.38777878e-17]
 [ 1.40800393e-04  1.88498690e-05]
 [ 9.77237544e-06  3.15797733e-05]
 [-1.38777878e-17  1.38777878e-17]
 [-1.38777878e-17  1.38777878e-17]
 [-1.38777878e-17  1.38777878e-17]
 [ 3.73888645e-06  3.64733194e-06]
 [ 4.69920023e-06  5.43694027e-06]
 [-4.86586185e-06  3.22986887e-06]
 [-1.38777878e-17  1

In [11]:
features = pd.read_csv('data/encoded_columns_name.txt', delimiter='\t', header=None)

In [12]:
features.columns = ['feature_name']

In [13]:
features['delta_mse'] = errperformance[:,0]
features['std_mse_change'] = errperformance[:,1]
features = features.sort_values('delta_mse', axis=0, ascending=False)
features

,feature_name,delta_mse,std_mse_change
427,AU_exchange,1.130145e+00,9.109565e-02
439,AU_eslt,8.707437e-01,7.147144e-02
436,AU_outlook,4.031963e-01,3.610932e-02
440,AU_officeclient,2.211478e-01,1.749643e-02
370,PaidEXOSeats,1.438092e-01,1.245429e-01
431,AU_od4b,1.181884e-01,9.309256e-03
329,PaidCount,1.178942e-01,7.322252e-02
487,FirstPaidTeamsStartDate,8.765640e-02,4.680851e-03
413,AU_exchange_05,7.214066e-02,3.634087e-03
444,HasSfB,6.245328e-02,1.147596e-02


In [14]:
len(features)

489

## Look at some examples on dev set

In [15]:
outputs_name = [
     'AR_exchange_06',
 'AR_sharepoint_06',
 'AR_skype_06',
 'AR_teams_06',
 'AR_od4b_06',
 'AR_onenote_06',
 'AR_word_06',
 'AR_excel_06',
 'AR_powerpoint_06',
 'AR_outlook_06',
 'AR_eslt_06',
 'AR_officeclient_06'
]

In [16]:
Xdev_show = Xdev[:3]
ydev_show = ydev[:3]
predictions = loaded_model.predict(Xdev_show)
# print('predictions shape:', predictions.shape)
data = np.concatenate((ydev_show,predictions), axis=0)
data_df = pd.DataFrame(data=data, columns=outputs_name)
data_df

,AR_exchange_06,AR_sharepoint_06,AR_skype_06,AR_teams_06,AR_od4b_06,AR_onenote_06,AR_word_06,AR_excel_06,AR_powerpoint_06,AR_outlook_06,AR_eslt_06,AR_officeclient_06
0,0.734375,0.000000,0.000000,0.000000,0.000000,0.000000,0.047222,0.030556,0.002778,0.297222,0.652778,0.297222
1,0.887500,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.012500,0.000000,0.062500,0.887500,0.075000
2,1.375000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.375000,0.000000
3,0.736666,0.000647,-0.001815,0.017586,0.018733,0.006842,0.015055,0.009503,0.009471,0.283905,0.735242,0.278910
4,0.756156,-0.000063,-0.005246,0.012503,0.066448,0.007137,-0.008417,-0.009617,0.002480,0.066241,0.762578,0.066907
5,1.396505,0.013625,0.003225,-0.001619,-0.017340,-0.002784,0.009152,-0.039731,-0.013593,0.036768,1.303271,0.015036


In [17]:
for i in range(len(ydev_show)):
    print('This is for dev example {}'.format(i))
    
    n = Xdev_show.shape[1]
    predictions = loaded_model.predict(Xdev_show[i].reshape((1, n)))
    
    for j in range(len(ydev_show[i])):
        print('{}_true: {}'.format(outputs_name[j], ydev_show[i][j]))
        print('{}_pred: {}'.format(outputs_name[j], predictions[0][j]))
        print('='*50)
    
    print('*'*20)

This is for dev example 0
AR_exchange_06_true: 0.734375
AR_exchange_06_pred: 0.7366665005683899
AR_sharepoint_06_true: 0.0
AR_sharepoint_06_pred: 0.0006472542881965637
AR_skype_06_true: 0.0
AR_skype_06_pred: -0.0018151979893445969
AR_teams_06_true: 0.0
AR_teams_06_pred: 0.01758648082613945
AR_od4b_06_true: 0.0
AR_od4b_06_pred: 0.018733076751232147
AR_onenote_06_true: 0.0
AR_onenote_06_pred: 0.006842270493507385
AR_word_06_true: 0.04722222222222222
AR_word_06_pred: 0.0150546133518219
AR_excel_06_true: 0.03055555555555556
AR_excel_06_pred: 0.009503096342086792
AR_powerpoint_06_true: 0.0027777777777777783
AR_powerpoint_06_pred: 0.009470563381910324
AR_outlook_06_true: 0.2972222222222222
AR_outlook_06_pred: 0.2839052677154541
AR_eslt_06_true: 0.6527777777777778
AR_eslt_06_pred: 0.7352428436279297
AR_officeclient_06_true: 0.2972222222222222
AR_officeclient_06_pred: 0.27891072630882263
********************
This is for dev example 1
AR_exchange_06_true: 0.8875
AR_exchange_06_pred: 0.756155908